In [1]:
import sqlite3

from IPython.core.magic import register_cell_magic

from tabulate import tabulate
import sqlparse

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

def run_and_print(sql):
    try:
        cursor.execute(sql)
    except sqlite3.OperationalError as e:
        print(sql)
        raise e

    conn.commit()

    rows = cursor.fetchall()
    if cursor.description is None:
        return

    print(tabulate(rows, headers=[d[0] for d in cursor.description]), end='\n\n')

@register_cell_magic
def sql(_, cell):
    for sql in sqlparse.split(cell):
        run_and_print(sql)

In [2]:
%%sql

CREATE TABLE trace(trace TEXT);
INSERT INTO trace VALUES ('Reticulating splines...');

CREATE TABLE pc_table(pc INTEGER);
INSERT INTO pc_table VALUES (0);
INSERT INTO trace VALUES ('pc_table CREATEd');

CREATE TABLE clk_table(clk BOOLEAN);
INSERT INTO clk_table VALUES (0);
INSERT INTO trace VALUES ('clk_table CREATEd');

pragma recursive_triggers = 1;


In [3]:
%%sql
-- create a trigger that increments the value of pc by 1, up to a maximum of 5, whenever the value of pc is updated

DROP TRIGGER IF EXISTS clk_trigger;
CREATE TRIGGER clk_trigger
AFTER UPDATE OF clk ON clk_table
BEGIN
    INSERT INTO trace VALUES (
        CASE
            WHEN OLD.clk = 0 AND NEW.clk = 1 THEN 'rising edge'
            WHEN OLD.clk = 1 AND NEW.clk = 0 THEN 'falling edge'
            ELSE 'no edge detected'  -- Optional, for other cases
        END
    );
END;

In [4]:
%%sql
DELETE FROM trace;

In [5]:
%%sql
-- drive the clock signal
UPDATE clk_table SET clk = 1;
UPDATE clk_table SET clk = 0;
UPDATE clk_table SET clk = 1;
UPDATE clk_table SET clk = 0;

In [6]:
%%sql
-- retrieve and print the logs
SELECT * FROM trace;

trace
------------
rising edge
falling edge
rising edge
falling edge

